# Reading Text From PDF File 

In [1]:
!pip install PyPDF2

In [0]:
## Extract Text from PDF File

def ExtractTextFromPDF(filePath, page):
  # importing required modules
  import PyPDF2 
  
  # creating a pdf file object 
  pdfFileObj = open(filePath, 'rb') 
  
  # creating a pdf reader object 
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
  
  # printing number of pages in pdf file 
  print(pdfReader.numPages) 
  
  # creating a page object 
  pageObj = pdfReader.getPage(page) 
  
  # extracting text from page 
  text = pageObj.extractText()
  
  # closing the pdf file object 
  pdfFileObj.close()

  # returning text
  return text

In [3]:
articlePath = '/content/drive/My Drive/Colab Notebooks/SemanticWeb/The_Semantic_Web_A_New_Form_of_Web_Content_That_is.pdf'
articleContent = ExtractTextFromPDF(articlePath, 1)

4


In [4]:
print(articleContent)
print('Content is ready..')

Scientific American: The Semantic Webhttp://www.sciam.com/print_version.cfm?articleID=00048144-10D2...1 of 329/07/2005 8.55ADVERTISEMENT BY MIGUEL SALMERONBY MIGUEL SALMERONWEB SEARCHES TODAY  May 17, 2001The Semantic WebA new form of Web content that is meaningful to computers will unleash a revolution of new possibilitiesBy Tim Berners-Lee, James Hendler and Ora LassilaThe entertainment system was belting out the Beatles' "We Can Work It Out" when the phone rang. When Peteanswered, his phone turned the sound down by sending a message to all the other local devices that had a volume control. His sister, Lucy, was on the line from the doctor's office: "Mom needs to see a specialist and then has to have a series of physical therapy sessions. Biweekly or something. I'm going to have my agent setup the appointments." Pete immediately agreed to share the chauffeuring.At the doctor's office, Lucy instructed her Semantic Web agent through her handheld Web browser. The agent promptly retrieve

# Parsing File

In [5]:
!pip install spacy
!python -m spacy download en_core_web_lg

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [6]:
!python -m spacy download en_core_web_lg

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 183, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.6/runpy.py", line 142, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.6/runpy.py", line 109, in _get_module_details
    __import__(pkg_name)
  File "/usr/local/lib/python3.6/dist-packages/spacy/__init__.py", line 12, in <module>
    from .cli.info import info as cli_info
  File "/usr/local/lib/python3.6/dist-packages/spacy/cli/__init__.py", line 6, in <module>
    from .train import train  # noqa: F401
  File "/usr/local/lib/python3.6/dist-packages/spacy/cli/train.py", line 16, in <module>
    from .._ml import create_default_optimizer
  File "/usr/local/lib/python3.6/dist-packages/spacy/_ml.py", line 31, in <module>
    import torch.nn
  File "/usr/local/lib/python3.6/dist-packages/torch/__init__.py", line 81, in <module>
    from torc

In [0]:
import spacy

In [0]:
#spacy.prefer_gpu()
nlp = spacy.load("en_core_web_lg")

In [0]:
# Keyword information from DB Pedia
def getDBPediaInfo(keyword):
  import requests
  dbPediaUrl = 'http://lookup.dbpedia.org/api/search.asmx/KeywordSearch?QueryString=' + keyword
  try:
    r = requests.get(url = dbPediaUrl, headers= {'Accept':'application/json'})
  except:
    print ('An error occured while sending request..')
    raise

  return r.json()

In [0]:
def parseDbPediaInfo(dbPediaResultObject):
  dbPediaObject = {}
  dbPediaObject['Count'] = 1

  if (len(dbPediaResultObject) < 1):
    return dbPediaObject

  dbPediaResult = dbPediaResultObject[0]
  dbPediaObject['Label'] = dbPediaResult['label']
  dbPediaObject['Uri'] = dbPediaResult['uri']
  dbPediaObject['Description'] = dbPediaResult['description']

  classes = []
  for dbPediaClass in dbPediaResult['classes']:
    classObj = {}
    classObj['Label'] = dbPediaClass['label']
    classObj['Uri'] = dbPediaClass['uri']
    classes.append(classObj)
  dbPediaObject['Classes'] = classes

  return dbPediaObject

In [0]:
# Print DB Pedia Result Object
def printResult(result):
  print('Label: ' + result['label'])
  print('Uri: ' + result['uri'])
  print('Description: ' + result['description'])
  print('Classes: ')
  for dbPediaClass in result['classes']:
    print ('\tClass Label: ' + dbPediaClass['label'])
    print ('\tClass Uri: ' + dbPediaClass['uri'])

In [12]:
# Test Result
testKeyword = 'The Beatles'
resultObj = getDBPediaInfo(testKeyword)

if(len(resultObj['results']) > 0):
  dbPediaResult = resultObj['results'][0]
  printResult(dbPediaResult)
else: 
  print ('There is no content about ' + testKeyword)

Label: The Beatles
Uri: http://dbpedia.org/resource/The_Beatles
Description: The Beatles were an English rock band formed in Liverpool in 1960 and one of the most commercially successful and critically acclaimed acts in the history of popular music. The group's best-known lineup consisted of John Lennon (rhythm guitar, vocals), Paul McCartney (bass guitar, vocals), George Harrison (lead guitar, vocals) and Ringo Starr (drums, vocals).
Classes: 
	Class Label: owl#Thing
	Class Uri: http://www.w3.org/2002/07/owl#Thing
	Class Label: organization
	Class Uri: http://schema.org/Organization
	Class Label: band
	Class Uri: http://dbpedia.org/ontology/Band
	Class Label: music group
	Class Uri: http://schema.org/MusicGroup
	Class Label: organisation
	Class Uri: http://dbpedia.org/ontology/Organisation
	Class Label: agent
	Class Uri: http://dbpedia.org/ontology/Agent


In [0]:
# Initialize variables

results = {}

In [0]:
doc = nlp(articleContent)
for token in doc:
  if token.text not in results:
    dbPediaResultObj = getDBPediaInfo(token.text)
    results[token.text] = parseDbPediaInfo(dbPediaResultObj['results'])
  else:
    results[token.text]['Count'] = results[token.text]['Count'] + 1

In [15]:
print (results)

{'Scientific': {'Count': 1, 'Label': 'UNESCO', 'Uri': 'http://dbpedia.org/resource/UNESCO', 'Description': 'The United Nations Educational, Scientific and Cultural Organization is a specialized agency of the United Nations (UN). Its stated purpose is to contribute to peace and security by promoting international collaboration through education, science, and culture in order to further universal respect for justice, the rule of law, and human rights along with fundamental freedoms proclaimed in the UN Charter.', 'Classes': []}, 'American': {'Count': 1, 'Label': 'United States', 'Uri': 'http://dbpedia.org/resource/United_States', 'Description': 'United States of America Flag of the United States Great Seal of the United States Motto:\xa0In God We Trust\xa0\xa0E Pluribus Unum\xa0\xa0 Anthem:\xa0"The Star-Spangled Banner"File:Star Spangled Banner instrumental. ogg Location of the United States CapitalWashington, D.C.', 'Classes': [{'Label': 'populated place', 'Uri': 'http://dbpedia.org/ont

# Save Data as JSON

In [0]:
import json

In [0]:
# Sort results by count
def sortResultsByCount(results):
  sortedResults = {}
  for item in sorted(results, key = lambda name: results[name]['Count'], reverse=True):
    sortedResults[item] = results[item]
  #print(sorted(results, key = lambda i: i['Count'], reverse=True))
  #print sorted(lis, key = lambda i: i['age'],reverse=True) 
  return sortedResults

In [0]:
exportPath = 'data.txt'

In [0]:
with open(exportPath, 'w') as outfile:
    json.dump(sortResultsByCount(results), outfile)

print('Document saved as ' + exportPath)